## 1. Cargar Librerías

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf

## 2. Cargar Datasets

In [2]:
df_books = pd.read_csv('clean_datasets/books.csv')
df_ratings = pd.read_csv('clean_datasets/ratings.csv')
df_to_read = pd.read_csv('clean_datasets/to_read.csv')

## 3. Preprocesar datos para el entrenamiento y evaluación del modelo

La siguiente función preprocesa los datos de libros, generando una matriz de características que se utilizará en el modelo de recomendaciones. Este proceso incluye la creación de embeddings para los autores, la transformación de etiquetas de los libros usando TF-IDF, y la normalización de las fechas de publicación.

#### Argumentos:
- `books_df (pd.DataFrame)`: DataFrame que contiene la información de los libros, como título, autores, etiquetas y fechas de publicación.
- `tag_weight (float)`: Un factor de ponderación para ajustar la importancia de las etiquetas en la matriz de características. Por defecto, se establece en 1.4.

#### Devuelve:
- `X (np.array)`: Matriz de características de los libros que contiene las siguientes representaciones concatenadas:
    - **Embeddings de los autores**: Representaciones binarizadas de los autores.
    - **Valoración promedio del libro**: `average_rating`.
    - **Año de publicación normalizado**: Una versión escalada del año de publicación.
    - **Matriz TF-IDF de las etiquetas (tags)**: Representación numérica de las etiquetas asociadas a cada libro, ajustada por el `tag_weight`.

#### Descripción del proceso:

1. **Embeddings de autores**:
   - Se crean embeddings binarios para los autores de los libros. Cada autor se asigna a una posición única en el vector y se le asigna un valor de 1 si el autor está asociado al libro.

2. **Preprocesamiento de etiquetas (tags)**:
   - Las etiquetas de los libros (`tag_name`) se procesan utilizando el modelo de TF-IDF (`TfidfVectorizer`), lo que genera una matriz TF-IDF que asigna un peso a cada etiqueta según su frecuencia relativa. El peso de las etiquetas se escala mediante el parámetro `tag_weight`.

3. **Normalización de fechas de publicación**:
   - El año de publicación original de cada libro se normaliza a un rango entre 0 y 1. Esto ayuda a estandarizar los valores en la matriz de características, asegurando que las fechas no dominen las demás características.

4. **Concatenación de características**:
   - Finalmente, se concatenan las siguientes matrices en una matriz de características completa (`X`):
     - Embeddings de los autores.
     - Valoración promedio del libro y año normalizado.
     - Matriz TF-IDF de etiquetas, escalada por el peso de las etiquetas.

Este preprocesamiento asegura que las características estén en un formato adecuado para ser utilizadas por el modelo de recomendaciones, y captura información relevante sobre los libros, incluyendo autores, popularidad, fecha de publicación y etiquetas.

In [3]:
def preprocess_data(books_df, tag_weight=1.4):
    # Crear embeddings de autores
    unique_authors = books_df['authors'].unique()
    author_to_index = {author: idx for idx, author in enumerate(unique_authors)}
    author_embeddings = np.zeros((len(books_df), len(unique_authors)))
    
    for i, author in enumerate(books_df['authors']):
        author_embeddings[i, author_to_index[author]] = 1.0
        
    # Procesar etiquetas con TF-IDF
    books_df['tag_name'].fillna('', inplace=True)
    tfidf = TfidfVectorizer(stop_words=None)
    tags_tfidf_matrix = tfidf.fit_transform(books_df['tag_name']).toarray() * tag_weight
    
    # Normalizar las fechas de publicación
    books_df['year_normalized'] = (books_df['original_publication_year'] - books_df['original_publication_year'].min()) / (
        books_df['original_publication_year'].max() - books_df['original_publication_year'].min())
    
    # Concatenar todas las características (embeddings de autores, calificaciones, fechas y TF-IDF de etiquetas)
    X = np.hstack([
        author_embeddings,
        books_df[['average_rating', 'year_normalized']].values,
        tags_tfidf_matrix
    ])
    
    return X

## 4. Construcción de la red neuronal siamesa

La siguiente función construye un modelo Siamese utilizando una red neuronal densa que compara dos entradas (representando dos libros) y calcula la distancia entre sus representaciones (embeddings). Este modelo es útil en tareas como recomendaciones, donde se quiere medir la similitud entre dos elementos.

#### Argumentos:
- `input_shape (tuple)`: Dimensión de la entrada de características de cada libro. Es un `tuple` que especifica el número de características que describen cada libro.

#### Devuelve:
- `tf.keras.Model`: Un modelo Siamese de Keras con dos entradas y una salida. La salida es la distancia euclidiana entre los embeddings de las dos entradas (libros). El modelo se compila con el optimizador Adam y usa el error cuadrático medio (`mean_squared_error`) como función de pérdida.

#### Descripción:

1. **Modelo base compartido**: 
   - El modelo `base_model` es una red secuencial de tres capas densas con activación ReLU. Este modelo extrae las representaciones de los libros en un espacio de características de menor dimensión. El modelo es compartido entre ambas entradas, lo que significa que los pesos de las capas son iguales para ambas entradas.

2. **Entradas del modelo**: 
   - La función toma dos entradas (`input_1` y `input_2`), que representan dos libros diferentes, cada uno con las mismas características (de tamaño `input_shape`).

3. **Representación de las entradas**: 
   - Cada entrada pasa por el mismo modelo base (`base_model`) para obtener sus respectivas representaciones (`encoded_1` y `encoded_2`).

4. **Cálculo de la distancia**: 
   - La distancia entre las dos representaciones es calculada usando la distancia euclidiana. Esto se realiza restando los embeddings de los dos libros y sumando el cuadrado de las diferencias por cada dimensión.

5. **Compilación del modelo**:
   - El modelo se compila con el optimizador Adam y se usa el error cuadrático medio (`mean_squared_error`) como la función de pérdida. Además, el modelo mide el error absoluto medio (`mae`) como métrica durante el entrenamiento.

Este enfoque permite entrenar el modelo para aprender las representaciones de los libros en un espacio común y medir su similitud a través de la distancia euclidiana.


In [4]:
def build_siamese_model(input_shape):
    # Modelo base que compartirá pesos
    base_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu')
    ])
    
    # Entradas para dos libros
    input_1 = tf.keras.layers.Input(shape=input_shape)
    input_2 = tf.keras.layers.Input(shape=input_shape)

    # Extraemos las representaciones usando el modelo base
    encoded_1 = base_model(input_1)
    encoded_2 = base_model(input_2)

    # Cálculo de la distancia euclidiana entre las dos representaciones
    distance = tf.keras.layers.Lambda(lambda embeddings: tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=1))([encoded_1, encoded_2])
    
    # Modelo completo con dos entradas y una salida
    siamese_model = tf.keras.Model(inputs=[input_1, input_2], outputs=distance)
    
    # Configurar el optimizador Adam con la precisión mixta
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    siamese_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    
    return siamese_model

## 5. Filtros para recomendaciones

Estas funciones se encargan de filtrar las recomendaciones de libros para evitar duplicados, colecciones, y mejorar la diversidad de las recomendaciones, controlando el número de libros del mismo autor y priorizando los primeros libros de series.

#### Función: `is_first_book_in_series`

Esta función determina si un libro es el primer volumen de una serie.

##### Argumentos:
- `title (str)`: El título del libro, que potencialmente contiene un indicador de su posición en una serie (por ejemplo, "#1").

##### Retorna:
- `bool`: 
   - `True` si el libro es el primer volumen de una serie (por ejemplo, tiene "#1" en el título).
   - `False` si el título contiene un número de volumen diferente (por ejemplo, "#2" o "#13").
   - Si no se encuentra un número en el título, se asume que el libro es el primero de la serie por defecto.

##### Descripción:
- Usa expresiones regulares para buscar la indicación de "volumen 1" en el título de un libro.
- Se excluyen los libros que tienen otros números como "#13", "#14", o rangos como "#1-5" para evitar confusiones con otros volúmenes de la serie.

In [5]:
def is_first_book_in_series(title):
    if re.search(r'#1\b(?![\.-])', title):
        return True
    elif re.search(r'#\d+', title):
        return False
    return True

#### Función: `filter_duplicate_titles`

Filtra títulos duplicados o aquellos que pertenecen a colecciones, como "box sets", trilogías, o ediciones completas.

##### Argumentos:
- `recommended_indices (list)`: Lista de índices de libros recomendados.
- `books_df (pd.DataFrame)`: DataFrame que contiene la información de los libros, incluyendo los títulos.

##### Retorna:
- `list`: Una lista filtrada de índices de libros, excluyendo aquellos que tienen palabras clave que indican colecciones o duplicados (por ejemplo, "box set", "trilogy", "omnibus").

##### Descripción:
- La función utiliza una lista de palabras clave asociadas con colecciones o ediciones especiales (por ejemplo, "box set", "complete collection").
- Para cada libro recomendado, verifica si su título contiene alguna de estas palabras clave y filtra los títulos que coincidan, devolviendo solo los libros individuales que no son colecciones.

In [6]:
def filter_duplicate_titles(recommended_indices, books_df):
    palabras_clave = ["box set", "boxset", "complete collection", "boxed set", "omnibus", "trilogy", "quartet", "quintet"]
    filtered_recommendations = []

    for idx in recommended_indices:
        title = books_df.iloc[idx]['title']
        if not any(keyword in title.lower() for keyword in palabras_clave):
            filtered_recommendations.append(idx)

    return filtered_recommendations

#### Función: `apply_diversity_filter`

Aplica un filtro de diversidad para asegurar que las recomendaciones no incluyan demasiados libros del mismo autor, y prioriza los primeros libros de las series.

##### Argumentos:
- `recommended_indices (list)`: Lista de índices de libros recomendados.
- `books_df (pd.DataFrame)`: DataFrame con la información de los libros, incluyendo autores y títulos.
- `book_id (int)`: ID del libro original sobre el cual se basó la recomendación, que debe ser excluido de los resultados.

##### Retorna:
- `list`: Una lista filtrada de índices que asegura una mayor diversidad en los autores y prioriza los primeros libros de las series.

##### Descripción:
- La función elimina el libro sobre el cual se basaron las recomendaciones de la lista de resultados.
- Usa un diccionario para llevar un registro del número de recomendaciones por autor, permitiendo hasta un máximo de 3 libros por autor.
- Además, prioriza la recomendación de los primeros libros de una serie utilizando la función `is_first_book_in_series`.

In [7]:
def apply_diversity_filter(recommended_indices, books_df, book_id):
    filtered_recommendations = []
    authors_seen = {}

    # Eliminar titulo sobre el que se hizo la recomendación
    recommended_indices = [idx for idx in recommended_indices if idx != book_id]

    for idx in recommended_indices:
        author = books_df.iloc[idx]['authors']
        title = books_df.iloc[idx]['title']

        if authors_seen.get(author, 0) < 3:
            if is_first_book_in_series(title):
                filtered_recommendations.append(idx)
                authors_seen[author] = authors_seen.get(author, 0) + 1

    return filtered_recommendations

## 6. Obtención de recomendaciones

Estas funciones permiten obtener el índice de un libro en un `DataFrame` y generar recomendaciones de libros similares utilizando un modelo Siamese para calcular distancias entre representaciones de libros (embeddings).

#### Función: `get_book_index_by_id`

Esta función devuelve el índice de un libro en el `DataFrame` de libros utilizando su `book_id`.

##### Argumentos:
- `book_id (int)`: ID único del libro que se desea buscar.
- `books_df (pd.DataFrame)`: DataFrame que contiene la información de los libros.

##### Retorna:
- `int`: El índice del libro en el `DataFrame`.

##### Excepciones:
- `ValueError`: Se lanza si el `book_id` proporcionado no se encuentra en el `DataFrame`.

##### Descripción:
- La función busca el libro con el `book_id` especificado en el `DataFrame`. Si no encuentra el libro, lanza una excepción `ValueError` con un mensaje indicando que no se pudo encontrar el libro.

In [8]:
def get_book_index_by_id(book_id, books_df):
    try:
        return books_df.loc[books_df['book_id'] == book_id].index[0]
    except IndexError:
        raise ValueError(f"El book_id {book_id} no se encontró en el DataFrame")

#### Función: `get_recommendations_with_batches`

Esta función genera una lista de recomendaciones de libros similares, calculando las distancias entre el libro dado y otros libros en lotes (batches), utilizando un modelo Siamese.

##### Argumentos:
- `book_id (int)`: ID del libro base para el cual se desean generar recomendaciones.
- `X (np.array)`: Matriz de características de los libros.
- `model (tf.keras.Model)`: El modelo Siamese que se utiliza para calcular las distancias entre los libros.
- `books_df (pd.DataFrame)`: DataFrame que contiene la información de los libros.
- `batch_size (int)`: Tamaño de los lotes en los que se procesan las características de los libros para calcular las distancias.
- `rating (float)`: Calificación dada por el usuario al libro base, que se usa para ponderar las distancias.
- `top_n (int)`: Número de recomendaciones que se desean devolver.

##### Retorna:
- `list`: Una lista de tuplas que contiene el `book_id` del libro recomendado y la distancia calculada entre el libro base y el libro recomendado.

##### Descripción:
1. **Obtener índice del libro base**:
   - La función comienza obteniendo el índice del libro base en el `DataFrame` de libros mediante la función `get_book_index_by_id`.

2. **Cálculo de distancias**:
   - Se extrae el vector de características del libro base y se calcula la distancia entre este libro y los otros libros en la matriz `X` utilizando el modelo Siamese.
   - El cálculo de distancias se realiza en lotes (`batches`) para mejorar la eficiencia cuando hay muchos libros en el `DataFrame`. 
   - Las distancias se ponderan inversamente al rating que el usuario ha dado al libro base (a menor rating, mayor peso en la distancia).

3. **Filtrado de recomendaciones**:
   - Tras calcular las distancias, los índices de los libros recomendados se ordenan en función de la distancia (de menor a mayor).
   - Se aplican dos filtros adicionales:
     - **Filtro de duplicados**: Se eliminan libros que son colecciones o tienen títulos duplicados usando `filter_duplicate_titles`.
     - **Filtro de diversidad**: Se asegura la diversidad de autores en las recomendaciones usando `apply_diversity_filter`, que limita el número de libros recomendados por autor.

4. **Formato de salida**:
   - La función devuelve una lista de las `top_n` recomendaciones en forma de tuplas, donde cada tupla contiene el `book_id` del libro recomendado y su distancia respecto al libro base.

In [9]:
def get_recommendations_with_batches(book_id, X, model, books_df, batch_size, rating, top_n):

    book_index = get_book_index_by_id(book_id, books_df)
    book_vector = X[book_index].reshape(1, -1)
    distances = []
    weight = 1.0/rating

    # Calcular distancias por lotes
    for i in range(0, len(X), batch_size):
        batch = X[i:i+batch_size]
        batch_size_actual = len(batch)
        book_batch = np.tile(book_vector, (batch_size_actual, 1))
        batch_distances = model.predict([book_batch, batch])
        distances.extend(batch_distances*weight)

    distances = np.array(distances).flatten()
    recommended_indices = distances.argsort()

    # Aplicar filtros de duplicados y diversidad
    filtered_recommendations = filter_duplicate_titles(recommended_indices, books_df)
    final_recommendations = apply_diversity_filter(filtered_recommendations, books_df, book_id=book_index)

    # Crear lista de tuplas (book_id, distancia)
    recommendations_with_distances = [(books_df.iloc[idx]['book_id'], distances[idx]) for idx in final_recommendations]

    return recommendations_with_distances[:top_n]

## 7. Proceso completo

In [10]:
# Cargar y preprocesar los datos
X = preprocess_data(df_books, tag_weight=1.4)

# Construir y entrenar el modelo siamesa
content_model = build_siamese_model(X.shape[1])

# Entrenar el modelo asegurando que se utilice la GPU
with tf.device('/GPU:0'):  
    content_model.fit([X, X], np.zeros(len(X)), epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
249/249 [==============================] - 2s 4ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/10
249/249 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 3/10
249/249 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 4/10
249/249 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 5/10
249/249 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 6/10
249/249 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 7/10
249/249 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - mae: 0.0000e+00 - val_los

In [11]:
# Seleccionar las valoraciones de un usuario específico
user_ratings = df_ratings[df_ratings['user_id'] == 1234]

# Obtener los libros calificados por el usuario
rated_books = user_ratings[['book_id', 'rating']]

# Realizar recomendaciones para cada libro calificado
recommended_books = []

for _, row in rated_books.iterrows():
    book_id = row['book_id']
    rating = row['rating']
    with tf.device('/GPU:0'):
        recommended_books_indices = get_recommendations_with_batches(
            book_id, X, content_model, df_books, 1048, rating, top_n=10
        )
    recommended_books.extend(recommended_books_indices)

# Preparar las recomendaciones en el formato correcto
recommended_books = pd.merge(pd.DataFrame(recommended_books, columns=['book_id', 'distance']), df_books, on='book_id')
recommended_books = recommended_books.sort_values('distance').drop_duplicates(subset=['book_id'], keep='first')
recommended_books = recommended_books[~recommended_books['book_id'].isin(user_ratings['book_id'])]
recommended_books = recommended_books['title']


16/16 [==============================] - 0s 1ms/step


In [12]:
# Visualizar las recomendaciones
print("Recomendaciones para el usuario 1:")
for book in recommended_books[:10]:
    print("-", book)

Recomendaciones para el usuario 1:
- Hide and Seek
- Suzanne's Diary for Nicholas
- 1st to Die (Women's Murder Club, #1)
- Blood Work (Harry Bosch Universe, #8; Terry McCaleb #1)
- Chasing the Dime
- Void Moon
- Murder at the Vicarage (Miss Marple, #1)
- The Seven-Percent Solution
- Don't Blink
- The Ice Princess (Patrik Hedström, #1)


## 8. Guardar el Modelo

In [13]:
content_model.save('recommender_model.h5')